In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np

In [6]:
data = np.array([
    [0.1, 7.4, 0],
    [-0.2, 5.3, 0],
    [0.2, 8.2, 1],
    [0.2, 7.7, 1]])
data

array([[ 0.1,  7.4,  0. ],
       [-0.2,  5.3,  0. ],
       [ 0.2,  8.2,  1. ],
       [ 0.2,  7.7,  1. ]])

In [8]:
loader = DataLoader(data, batch_size=2, shuffle=False)
batch = next(iter(loader))
batch

tensor([[ 0.1000,  7.4000,  0.0000],
        [-0.2000,  5.3000,  0.0000]], dtype=torch.float64)

加载器从数据集中选择了 2 个样本。 

这些样本被转换为张量（2 个大小为 3 的样本）。

创建并返回一个新的张量 (2x3)。

In [14]:
from pprint import pprint
# now dataset is a list of dicts
dict_data = [
    {'x1': 0.1, 'x2': 7.4, 'y': 0},
    {'x1': -0.2, 'x2': 5.3, 'y': 0},
    {'x1': 0.2, 'x2': 8.2, 'y': 1},
    {'x1': 0.2, 'x2': 7.7, 'y': 10},
]
pprint(dict_data)
dict_data

[{'x1': 0.1, 'x2': 7.4, 'y': 0},
 {'x1': -0.2, 'x2': 5.3, 'y': 0},
 {'x1': 0.2, 'x2': 8.2, 'y': 1},
 {'x1': 0.2, 'x2': 7.7, 'y': 10}]


[{'x1': 0.1, 'x2': 7.4, 'y': 0},
 {'x1': -0.2, 'x2': 5.3, 'y': 0},
 {'x1': 0.2, 'x2': 8.2, 'y': 1},
 {'x1': 0.2, 'x2': 7.7, 'y': 10}]

In [24]:
loader = DataLoader(dict_data, batch_size=2, shuffle=False)
batch = next(iter(loader))
batch

{'x1': tensor([ 0.1000, -0.2000], dtype=torch.float64),
 'x2': tensor([7.4000, 5.3000], dtype=torch.float64),
 'y': tensor([0, 0])}

In [25]:
nlp_data = [
    {'tokenized_input': [1, 4, 5, 9, 3, 2],
     'label':0},
    {'tokenized_input': [1, 7, 3, 14, 48, 7, 23, 154, 2],
     'label':0},
    {'tokenized_input': [1, 30, 67, 117, 21, 15, 2],
     'label':1},
    {'tokenized_input': [1, 17, 2],
     'label':0},
]
loader = DataLoader(nlp_data, batch_size=2, shuffle=False)
batch = next(iter(loader))

RuntimeError: each element in list of batch should be of equal size

错误消息表明不可能创建非矩形张量。 顺便说一句，可以看到触发错误的是 default_collate函数。

我们可以做什么？ 有两种解决方案：

1. 将整个数据集填充到最长的样本。

2. 在批创建期间动态填充。

第一个解决方案可能看起来更简单—只需将所有样本扩展到最长的样本即可。 但有一个问题—我们会浪费内存和计算能力（它们在 GPU 上很昂贵！）来处理 padding，这并不影响结果。 如果我们的数据中有一些长序列，而且大多数序列都相对较短，那就尤其痛苦。 在这种情况下，我们主要是处理填充而不是数据！

如果我们将整个数据集填充到最长的序列，会浪费大量空间！

另一种方法是动态填充数据。 当选择该批的样本时，我们只将它们填充到最长的样本。 如果我们另外按长度对数据进行排序，则填充将是最小的。 如果有一些非常长的序列，它们只会影响它们的批次，而不是整个数据集。

In [30]:
from torch.nn.utils.rnn import pad_sequence #(1)

def custom_collate(data): #(2)
    inputs = [torch.tensor(d['tokenized_input']) for d in data] #(3)
    labels = [d['label'] for d in data]

    inputs = pad_sequence(inputs, batch_first=True) #(4)
    labels = torch.tensor(labels) #(5)

    return { #(6)
        'tokenized_input': inputs,
        'label': labels
    }

loader = DataLoader(
  	nlp_data, 
    batch_size=2, 
    shuffle=False, 
    collate_fn=custom_collate
) #(7)

iter_loader = iter(loader)
batch1 = next(iter_loader)
pprint(batch1)
batch2 = next(iter_loader)
pprint(batch2)

{'label': tensor([0, 0]),
 'tokenized_input': tensor([[  1,   4,   5,   9,   3,   2,   0,   0,   0],
        [  1,   7,   3,  14,  48,   7,  23, 154,   2]])}
{'label': tensor([1, 0]),
 'tokenized_input': tensor([[  1,  30,  67, 117,  21,  15,   2],
        [  1,  17,   2,   0,   0,   0,   0]])}


代码说明如下：

我们使用 pad_sequence进行填充

Collate 函数要传入单个参数 - 样本列表。 在这种情况下，它将是一个字典列表，但它也可以是一个元组列表等——具体取决于数据集。

当数据出现时，如果格式为“字典列表”，我们需要遍历它并为所有输入和标签创建一个单独的列表。 与此同时， tokenized_input 被转换为一维张量（它是一个整数列表）。

执行填充。

由于标签是整数列表，我们将其转换为张量。

返回格式化的批次。

在加载器中设置我们的自定义整理函数。

正如我们所看到的，批的格式与字典的默认排序规则相同。 我们清楚地看到填充量很小。

In [32]:
nlp_data

inputs = [torch.tensor(d['tokenized_input']) for d in nlp_data] #(3)
labels = [d['label'] for d in nlp_data]

inputs = pad_sequence(inputs, batch_first=True) #(4)
labels = torch.tensor(labels) #(5)

In [44]:
from torch import tensor


inputs = [torch.tensor(d['tokenized_input']) for d in nlp_data]
inputs = pad_sequence(inputs, batch_first=True) #(4)
inputs
# torch.tensor(inputs)

tensor([[  1,   4,   5,   9,   3,   2,   0,   0,   0],
        [  1,   7,   3,  14,  48,   7,  23, 154,   2],
        [  1,  30,  67, 117,  21,  15,   2,   0,   0],
        [  1,  17,   2,   0,   0,   0,   0,   0,   0]])

In [53]:
nlp_data = [
    {'tokenized_input': [1, 4, 5, 9, 3, 2],
     'label':0},
    {'tokenized_input': [1, 7, 3, 14, 48, 7, 23, 154, 2],
     'label':0},
    {'tokenized_input': [1, 30, 67, 117, 21, 15, 2],
     'label':1},
    {'tokenized_input': [1, 17, 2],
     'label':0},
]
from torch.nn.utils.rnn import pad_sequence #(1)

def custom_collate(data): #(2)
    inputs = [torch.tensor(d['tokenized_input']) for d in data] #(3)
    labels = [d['label'] for d in data]

    inputs = pad_sequence(inputs, batch_first=True) #(4)
    labels = torch.tensor(labels) #(5)

    return { #(6)
        'tokenized_input': inputs,
        'label': labels
    }

loader = DataLoader(
  	nlp_data, 
    batch_size=2, 
    shuffle=False, 
    collate_fn=custom_collate
) #(7)

for i, (X, y) in enumerate(loader):
    print(i, type(X), type(y))  

0 <class 'str'> <class 'str'>
1 <class 'str'> <class 'str'>


In [51]:
iter_loader = iter(loader)
batch1 = next(iter_loader)
pprint(batch1)
batch2 = next(iter_loader)
pprint(batch2)

{'label': tensor([0, 0]),
 'tokenized_input': tensor([[  1,   4,   5,   9,   3,   2,   0,   0,   0],
        [  1,   7,   3,  14,  48,   7,  23, 154,   2]])}
{'label': tensor([1, 0]),
 'tokenized_input': tensor([[  1,  30,  67, 117,  21,  15,   2],
        [  1,  17,   2,   0,   0,   0,   0]])}
